<a href="https://colab.research.google.com/github/brandonb1121/personalprojects/blob/main/DataCollectionCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, date

In [2]:
#Dates only taken from Dec 01 to May 31 the following year
delta1 = timedelta(days=7)
d = date(2001, 12, 1)
dates = []
while d < date(2022, 3, 8):
  if d.month <= 5 or d.month >= 12:
    dates.append(int(d.strftime("%Y%m%d")))
  d += delta1
#Jan 29, 2011 doesn't have data so we are replacing it with Jan 28, 2011
dates.index(20110129)
dates[243] = 20110128
#Similarly, Feb 19, 2011 also doesn't contain any data so we are replacing it with Feb 18, 2011
dates.index(20110219)
dates[246] = 20110218

In [3]:
#Dec 2018 - Mar 2022
init_dates = []
for date in dates:
  if date >= 20181201:
    init_dates.append(date)

rows = []
for date in init_dates:
  response = requests.get("https://cdec.water.ca.gov/reportapp/javareports?name=PAGE6.%d" % date)
  soup = BeautifulSoup(response.content, "html.parser")
  tables = soup.find_all("body")
  table = tables[1]
  for row in table.find_all("tr")[1:]:
    cells = row.find_all("td")
    if len(cells) > 2:
      station = cells[0].text
      station_id = cells[1].text
      coop_agency = cells[2].text
      elev = cells[3].text
      apr_1_avg = cells[4].text
      today = cells[5].text
      per_apr_1 = cells[6].text
      yesterday = cells[7].text
      last_week = cells[8]
      rows.append({
          "station": station,
          "station_id": station_id,
          "coop_agency": coop_agency,
          "elev (FT)": elev,
          "apr_1_avg": apr_1_avg,
          "daily snowfall": today,
          "percent_apr_1": per_apr_1,
          "24_hrs_ago": yesterday,
          "1_wk_ago": last_week,
          "date": date
      })

In [4]:
df_earlier = pd.DataFrame(rows)
df_earlier

,station,station_id,coop_agency,elev (FT),apr_1_avg,daily snowfall,percent_apr_1,24_hrs_ago,1_wk_ago,date
0,Peterson Flat,PET,DWR,"7,150",29.2,5.4 r,18%,4.8 r,[2.8 r],20181201
1,Red Rock Mountain,RRM,DWR,"6,700",39.6,7.9 r,20%,7.4 r,[4.2 r],20181201
2,Bonanza King,BNK,DWR,"6,450",40.5,5.0,12%,4.7,[2.3 r],20181201
3,Shimmy Lake,SHM,DWR,"6,400",40.3,6.7,17%,6.3,[3.3 r],20181201
4,Middle Boulder 3,MB3,DWR,"6,200",28.3,4.8,17%,4.2,[2.2 r],20181201
...,...,...,...,...,...,...,...,...,...,...
12054,Cottonwood Lakes,CWD,DWR,"10,150",11.6,8.5,74%,8.5,[8.3 ],20220305
12055,Big Pine Creek,BGP,DWR,"9,800",17.9,12.2 r,68%,12.2 r,[12.0 r],20220305
12056,Rock Creek Lakes,RCK,DWR,"9,700",14.0,8.7 r,62%,8.9 r,[9.5 r],20220305
12057,South Lake,SLK,DWR/DFM/SMN,"9,600",16.0,9.8 r,61%,9.8 r,[10.2 r],20220305


In [5]:
#Dec 2001-May 2018
rows = []
other_dates = []
for date in dates:
  if date < 20181201:
    other_dates.append(date)
for date in other_dates:
  response = requests.get("https://cdec.water.ca.gov/reportapp/javareports?name=PAGE6.%d" % date)
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.find("table")
  for row in table.find_all("tr")[1:]:
    cells = row.find_all("td")
    if len(cells) > 2:
      station = cells[0].text
      station_id = cells[1].text
      coop_agency = cells[2].text
      elev = cells[3].text
      apr_1_avg = cells[4].text
      today = cells[5].text
      per_apr_1 = cells[6].text
      yesterday = cells[7].text
      last_week = cells[8]
      rows.append({
          "station": station,
          "station_id": station_id,
          "coop_agency": coop_agency,
          "elev (FT)": elev,
          "apr_1_avg": apr_1_avg,
          "daily snowfall": today,
          "percent_apr_1": per_apr_1,
          "24_hrs_ago": yesterday,
          "1_wk_ago": last_week,
          "date": date
      })

In [6]:
#removing all accidental rows that have been created while scraping the sites
df_older = pd.DataFrame(rows)
df_older.drop(df_older.index[df_older["station_id"] == "\xa0"], inplace=True)
df_older.drop(df_older.index[df_older["station"] == "Station "], inplace=True)
df_older.reset_index(inplace=True)
df_older.drop("index", axis=1, inplace=True)
df_older

,station,station_id,coop_agency,elev (FT),apr_1_avg,daily snowfall,percent_apr_1,24_hrs_ago,1_wk_ago,date
0,Peterson Flat,PET,DWR,"7,150",29.2,7.3,25%,5.6,[1.6 ],20011201
1,Red Rock Mountain,RRM,DWR,"6,700",39.6,11.8,29%,10.5,[4.6 ],20011201
2,Bonanza King,BNK,DWR,"6,450",40.5,6.2,15%,5.8,[0.8 ],20011201
3,Shimmy Lake,SHM,DWR,"6,400",40.3,10.9,27%,8.9,[2.9 ],20011201
4,Middle Boulder 3,MB3,DWR,"6,200",28.3,8.1r,28%,8.1,[0.9r],20011201
...,...,...,...,...,...,...,...,...,...,...
55931,Cottonwood Lakes,CWD,DWR,"10,150",11.6,0.0r,0%,0.0r,"[0.0, [r]]",20180526
55932,Big Pine Creek,BGP,DWR,"9,800",17.9,0.0r,0%,0.0r,"[0.0, [r]]",20180526
55933,Rock Creek Lakes,RCK,DWR,"9,700",14.0,0.0r,0%,0.0r,"[0.0, [r]]",20180526
55934,South Lake,SLK,DWR/SS,"9,600",16.0,0.0r,0%,0.0r,"[0.0, [r]]",20180526


In [77]:
#Cleaning of rows and columns
df_snowfall = pd.concat([df_older, df_earlier])
df_snowfall.reset_index(inplace=True)
df_snowfall.drop("index", axis=1, inplace=True)
df_snowfall.replace("---", None, inplace=True)
df_snowfall.replace("----", None, inplace=True)
df_snowfall['elev (FT)'] = pd.to_numeric(df_snowfall['elev (FT)'].str.replace(r'\D', ''))
df_snowfall["apr_1_avg"] = pd.to_numeric(df_snowfall["apr_1_avg"])
df_snowfall['daily snowfall'] = pd.to_numeric(df_snowfall['daily snowfall'].str.replace(r'\D', ''))/10
df_snowfall["percent_apr_1"] = pd.to_numeric(df_snowfall["percent_apr_1"].str[:-1])
df_snowfall['24_hrs_ago'] = pd.to_numeric(df_snowfall['24_hrs_ago'].str.replace(r'\D', ''))/10
df_snowfall["date"] = pd.to_datetime(df_snowfall["date"], format="%Y%m%d")
df_snowfall["prop_apr_1"] = df_snowfall["percent_apr_1"]/100
df_snowfall

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


,station,station_id,coop_agency,elev (FT),apr_1_avg,daily snowfall,percent_apr_1,24_hrs_ago,1_wk_ago,date,prop_apr_1
0,Peterson Flat,PET,DWR,7150,29.2,7.3,25,5.6,[1.6 ],2001-12-01,0.25
1,Red Rock Mountain,RRM,DWR,6700,39.6,11.8,29,10.5,[4.6 ],2001-12-01,0.29
2,Bonanza King,BNK,DWR,6450,40.5,6.2,15,5.8,[0.8 ],2001-12-01,0.15
3,Shimmy Lake,SHM,DWR,6400,40.3,10.9,27,8.9,[2.9 ],2001-12-01,0.27
4,Middle Boulder 3,MB3,DWR,6200,28.3,8.1,28,8.1,[0.9r],2001-12-01,0.28
...,...,...,...,...,...,...,...,...,...,...,...
67990,Cottonwood Lakes,CWD,DWR,10150,11.6,8.5,74,8.5,[8.3 ],2022-03-05,0.74
67991,Big Pine Creek,BGP,DWR,9800,17.9,12.2,68,12.2,[12.0 r],2022-03-05,0.68
67992,Rock Creek Lakes,RCK,DWR,9700,14.0,8.7,62,8.9,[9.5 r],2022-03-05,0.62
67993,South Lake,SLK,DWR/DFM/SMN,9600,16.0,9.8,61,9.8,[10.2 r],2022-03-05,0.61


In [11]:
stations = df_snowfall["station_id"].unique()
stations

array(['PET', 'RRM', 'BNK', 'SHM', 'MB3', 'HIG', 'SCT', 'MUM', 'BFL',
       'CWF', 'CDP', 'BLA', 'SDF', 'MED', 'ADM', 'SNM', 'SLT', 'STM',
       'LLP', 'KTL', 'GRZ', 'PLP', 'GOL', 'HMB', 'HRK', 'RTL', 'BKL',
       'FOR', 'NLS', 'SCN', 'LOS', 'CXS', 'CAP', 'ALP', 'FRN', 'MDW',
       'SIL', 'CSL', 'VVL', 'HYS', 'RCC', 'RBB', 'GKS', 'BLC', 'RBP',
       'DDM', 'HHM', 'GNL', 'REL', 'BLK', 'SLM', 'BLD', 'BLS', 'DAN',
       'SLI', 'TUM', 'HRS', 'STR', 'TNY', 'WHW', 'PDS', 'GIN', 'KIB',
       'VLC', 'AGP', 'KSP', 'GRM', 'DPO', 'TMR', 'CHM', 'HNT', 'GRV',
       'PSR', 'BSH', 'CRL', 'STL', 'BCB', 'MTM', 'UBC', 'WWC', 'BIM',
       'FRW', 'QUA', 'GNF', 'UTY', 'CBT', 'CHP', 'PSC', 'WTM', 'TUN',
       'CSV', 'BCH', 'DSS', 'BMW', 'IDP', 'SQV', 'IDC', 'INN', 'TK2',
       'MSK', 'HVN', 'HGM', 'MRL', 'EP5', 'RP2', 'TCC', 'WC3', 'FLL',
       'EBB', 'HOR', 'MNT', 'BSK', 'FDC', 'PSN', 'SPT', 'LVT', 'SDW',
       'VRG', 'LBD', 'SPS', 'LVM', 'GEM', 'SWM', 'CWD', 'BGP', 'RCK',
       'SLK', 'MHP',

In [12]:
rows = []
for station in stations:
  subrow = []
  response = requests.get("https://cdec.water.ca.gov/dynamicapp/staMeta?station_id=%s" % station)
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.find("table", {"border":1})
  for row in table.find_all("td"):
    subrow.append(row.text)
  rows.append({
      "station_id":subrow[1],
      "elevation":subrow[3],
      "river_basin":subrow[5],
      "county":subrow[7],
      "hydrologic_area":subrow[9],
      "latitude":subrow[13],
      "longitude":subrow[15]
  })

In [76]:
df_stations = pd.DataFrame(rows)
df_stations["latitude"] = df_stations["latitude"].str[:-1]
df_stations["latitude"] = pd.to_numeric(df_stations["latitude"])
df_stations["longitude"] = df_stations["longitude"].str[:-1]
df_stations["longitude"] = pd.to_numeric(df_stations["longitude"])
df_stations["elevation"] = pd.to_numeric(df_stations["elevation"].str[:-2])
df_stations

,station_id,elevation,river_basin,county,hydrologic_area,latitude,longitude
0,PET,7150,TRINITY R,TRINITY,NORTH COAST,41.302219,-122.527779
1,RRM,6700,TRINITY R,TRINITY,NORTH COAST,41.023621,-122.886627
2,BNK,6450,TRINITY R,TRINITY,NORTH COAST,41.083118,-122.631271
3,SHM,6400,TRINITY R,TRINITY,NORTH COAST,41.005299,-122.801598
4,MB3,6200,SCOTT R,SISKIYOU,NORTH COAST,41.224838,-122.806160
...,...,...,...,...,...,...,...
129,KUB,8073,KINGS R,FRESNO,TULARE LAKE,36.966900,-119.056600
130,KUP,5611,KINGS R,FRESNO,TULARE LAKE,37.061000,-119.183000
131,MDL,7900,MOKELUMNE R,AMADOR,SAN JOAQUIN RIVER,38.615000,-120.140000
132,TES,9945,TUOLUMNE R,TUOLUMNE,SAN JOAQUIN RIVER,37.910854,-119.258453


In [84]:
#csv file found from https://gist.github.com/Ewiseman/b251e5eaf70ca52a4b9b10dce9e635a4#file-ski_resort_stats-csv
df_ski = pd.read_csv("ski_resort_stats.csv")
df_ca_resorts = df_ski[df_ski["state"] == "California"]
df_ca_resorts

,resort_name,state,summit,base,vertical,lifts,runs,acres,green_percent,green_acres,blue_percent,blue_acres,black_percent,black_acres,lat,lon
2,"Alpine Meadows, CA",California,8673.0,6385.0,2288.0,13.0,100.0,2400,0.25,600.00,0.40,960.00,0.35,840.00,39.1700,-120.220
23,"Bear Mountain, CA",California,8805.0,7140.0,1665.0,12.0,NaN,748,0.30,224.40,0.40,299.20,0.30,224.40,37.1000,-119.100
25,"Bear Valley Mt, CA",California,8500.0,6600.0,1900.0,9.0,75.0,1680,0.25,420.00,0.40,672.00,0.35,588.00,39.2000,-122.440
47,"Boreal, CA",California,7700.0,7200.0,500.0,6.0,41.0,480,0.30,144.00,0.55,264.00,0.15,72.00,39.3300,-120.350
81,"China Peak, CA",California,8709.0,7030.0,1679.0,11.0,45.0,1200,0.11,132.00,0.28,336.00,0.61,732.00,40.9000,-123.340
102,"Dodge Ridge, CA",California,8200.0,6600.0,1600.0,12.0,67.0,852,0.20,170.40,0.40,340.80,0.40,340.80,38.1400,-120.030
103,"Donner Ski Ranch, CA",California,7781.0,7031.0,750.0,8.0,53.0,503,0.25,125.75,0.50,251.50,0.25,125.75,39.3100,-120.340
127,"Heavenly, CA",California,10067.0,6540.0,3527.0,29.0,97.0,4800,0.20,960.00,0.45,2160.00,0.35,1680.00,38.9353,-119.940
138,"Homewood, CA",California,7800.0,6230.0,1570.0,8.0,64.0,1260,0.15,189.00,0.50,630.00,0.35,441.00,39.0800,-120.170
151,"Kirkwood, CA",California,9800.0,7800.0,2000.0,15.0,81.0,2300,0.15,345.00,0.50,1150.00,0.35,805.00,38.6840,-120.068


In [83]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
path1 = '/content/drive/My Drive/df_snowfall.csv'
path2 = '/content/drive/My Drive/df_stations.csv'
path3 = '/content/drive/My Drive/df_ca_resorts.csv'

with open(path1, 'w', encoding = 'utf-8-sig') as f:
  df_snowfall.to_csv(f)
with open(path2, 'w', encoding = 'utf-8-sig') as f:
  df_stations.to_csv(f)
with open(path3, 'w', encoding = 'utf-8-sig') as f:
  df_ca_resorts.to_csv(f)

Mounted at /content/drive
